# Ceneo Scraper

## Struktura opinii w serwisie Ceneo.pl

|składowa|selektor|zmienna|
|--------|--------|-------|
|identyfikator opinii|["data-entry-id"]|opinion_id|
|autor|user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation > em|recommend|
|liczbę gwiazdek|span.user-post__score-count|stars|
|treść opinii|div.user-post__text|content|
|listę wad|review-feature__title--positives ~ div.review-feature__item|cons|
|listę zalet|review-feature__title--negatives ~ div.review-feature__item|pros|
|data wystawienia opinii|span.user-post__published > time:nth-child(1)["datetime"]|opinion_date|
|data zakupu produktu|span.user-post__published > time:nth-child(2)["datetime"]|purchase_date|
|ile osób uznało opinię za przydatną|button.vote-yes["data-total-vote"]|up_vote|
|ile osób uznało opinię za nieprzydatną|button.vote-no["data-total-vote"]|down_vote|

In [30]:
selectors = {
"opinion_id": (None,"data-entry-id"),
"author": (".user-post__author-name",),
"recommend": ("span.user-post__author-recomendation > em",),
"stars": ("span.user-post__score-count",),
"content": ("div.user-post__text",),
"cons": ("div.review-feature__title review-feature__title--positives ~ div.review-feature__item",None,True),
"pros": ("div.review-feature__title review-feature__title--negatives ~ div.review-feature__item",None,True),
"opinion_date": ("span.user-post__published > time:nth-child(1)","datetime"),
"purchase_date": ("span.user-post__published > time:nth-child(2)","datetime"),
"up_vote": ("button.vote-yes","data-total-vote"),
"down_vote": ("button.vote-no","data-total-vote")
}

### Funckja do ekstrakcji danych

In [31]:
def extract(ancestor, selector=None, attribute=None, return_list = False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()


### Import bibliotek

In [32]:
import os
import json
import requests
from bs4 import BeautifulSoup

### Wczytanie kodu produktu o którym pobierzemy opinie

In [33]:
#product_id = 104109736
product_id = input("Podaj kod produktu: ")
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"


### Wysłanie do serwera Ceneo.pl żądania dostępu do strony z opiniami produktu

In [34]:
response = requests.get(url)

### Przekształcenie kodu HTML w postachi tekstowej do obiektu reprezuntującego strukture DOM
### Wydobycie ze struktury DOM znaczników odpowiadająchc opiniom konsumentów

### Wydobycie ze struktury znacznika odpowiadającego pojedynczej opinii jej składowych

### Pobranie wszytskich opinii o wskazanym produkcie z serwisu Ceneo.pl

In [35]:
all_opinions = []
while(url):
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion,*value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:
        url = "https://www.ceneo.pl/" + extract(page, "a.pagination__next", "href")
    except TypeError:
        url = None


### Zapis opinii o produkcie do pliku json do katalogu opinions

In [36]:
if not os.path.exists("opinions"):
    os.mkdir('opinions')
with open(f'opinions/{product_id}.json', "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)

In [37]:
all_opinions

[{'opinion_id': '10884232',
  'author': 'Grzegorz',
  'recommend': 'Polecam',
  'stars': '4/5',
  'content': 'Głośnik daje radę jest lekki, ale szkoda że ma słaby akumulator. Ma w środku tyle miejsca nie można by dołożyć jeszcze jednego takiego akumulatorka!',
  'cons': [],
  'pros': [],
  'opinion_date': '2019-08-07 01:31:13',
  'purchase_date': '2019-05-04 12:18:41',
  'up_vote': '1',
  'down_vote': '0'},
 {'opinion_id': '10704162',
  'author': 'edyta',
  'recommend': None,
  'stars': '3,5/5',
  'content': 'jest ok ,myslalam ,ze bedzie troche mocniejsza ale za tę cene  ok ,fajnie ze mikrofon można sie powyglupiac',
  'cons': [],
  'pros': [],
  'opinion_date': '2019-06-26 11:09:20',
  'purchase_date': '2019-06-17 22:52:53',
  'up_vote': '1',
  'down_vote': '0'},
 {'opinion_id': '12551559',
  'author': 'Użytkownik Ceneo',
  'recommend': 'Polecam',
  'stars': '4/5',
  'content': 'Bardzo dobry dźwięk i miło się słucha muzyki dla nie wybrednych.',
  'cons': [],
  'pros': [],
  'opinion_d